#### *Demonstration purposes only. Change organization connections, variables, and item searches to match your criteria.* Code will only run on ArcGIS Enterprise deployments.

### Import Libraries

In [ ]:
from arcgis.gis import GIS, GroupMigrationManager

### Connect to Web GIS organizations

In [ ]:
source = GIS(profile="<source_enterprise_profile>", verify_cert=False)
target = GIS(profile="<target_enterprise_profile", verify_cert=False)

### Search for source org Group from which to export content

In [ ]:
ita_group = source.groups.search("Italy Environment Data")[0]
ita_group

In [ ]:
ita_group.content()

### Initialize Source GroupMigratonManager 

In [ ]:
grp_mgr = GroupMigrationManager(ita_group)

### Create Export Package Job

In [ ]:
ita_job = grp_mgr.create()

In [ ]:
import time

while ita_job.status != "completed":
    job_status = ita_job.status
    if job_status == "failed":
        break
    else:
        print(job_status)
        time.sleep(5)
print(f"Job Status: {ita_job.status}")

### Inspect Job Results

In [ ]:
ita_epk_item = ita_job.result()
ita_epk_item

In [ ]:
grp_mgr.inspect(ita_epk_item)

### Download the Export Package

In [ ]:
down_res = ita_epk_item.download(save_path=r"</file/path/on/system",
                                 file_name=r"ita_download.epk")
down_res

### Add Export Package to Target organization

In [ ]:
target_epk_item = target.content.add(item_properties={"title" : "ita_data_exp_pkg",
                                                      "tags" : "ITA, environment, hydrology",
                                                      "snippet": "Natural resource data for Italy.",
                                                      "type": "Export Package"},
                                     data=down_res)
target_epk_item

### Search for target org Group for which to import content

In [ ]:
target.users.me

In [ ]:
target_grp = target.groups.search("Italy Natural Environment")[0]
target_grp

In [ ]:
target_grp.id

### Initialize Target GroupMigrationManager

In [ ]:
target_grp_mgr = GroupMigrationManager(target_grp)

### Load the Group content into the Target

In [ ]:
target_epk_job = target_grp_mgr.load(epk_item=target_epk_item)

In [ ]:
import time

while target_epk_job.status != "completed":
    job_status = target_epk_job.status
    if job_status == "failed":
        break
    else:
        print(job_status)
        time.sleep(5)
print(f"Job Status: {target_epk_job.status}")

In [ ]:
target_epk_job.messages

In [ ]:
target_epk_job.result()